Download GNPS data <br>
Replace Job ID below with your GNPS job ID:

In [1]:
! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=b817262cb6114e7295fee4f73b22a3ad&view=download_cytoscape_data' -o GNPS_output_graphML.zip
! unzip -d GNPS_output_graphML/ GNPS_output_graphML.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2078k    0 2078k    0     0   140k      0 --:--:--  0:00:14 --:--:--  198k
Archive:  GNPS_output_graphML.zip
  inflating: GNPS_output_graphML/FEATURE-BASED-MOLECULAR-NETWORKING-b817262c-download_cytoscape_data-main.graphml  
  inflating: GNPS_output_graphML/params.xml  
  inflating: GNPS_output_graphML/DB_result/199547f600354368817ea480a44d4791.tsv  
  inflating: GNPS_output_graphML/networking_pairs_results_file_filtered/0aafc490cc974963a5b02dccc4cd27f6.tsv  
  inflating: GNPS_output_graphML/clusterinfo_summary/864b10ab978e40a68ddb30f358079b00.tsv  
  inflating: GNPS_output_graphML/gnps_molecular_network_graphml/e359f26d5ed4488eb589126f2fe9abc8.graphml  


specify your NAP job ID:

In [1]:
nap_id = 'c4bb6b8be9e14bdebe87c6ef3abe11f6'

load libraries

In [2]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = '/Applications/MarvinSuite/bin/' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [3]:
netfile = 'GNPS_output_graphML/clusterinfo_summary/' + str(os.listdir('GNPS_output_graphML/clusterinfo_summary/')[0])
gnpslibfile = 'GNPS_output_graphML/DB_result/'+ str(os.listdir('GNPS_output_graphML/DB_result/')[0])

In [4]:
nap = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t')

add all chemical structural information output as dataframe items in list

In [5]:
matches = [gnpslib, nap]

In [6]:
out = unique_smiles(matches)

In [7]:
out['df'].to_csv("SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [8]:
! molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys.txt

In [9]:
ikeys = pd.read_csv("InchiKeys.txt",  sep='\t',header = None)

out['df']["inchikey"] = ikeys
inchi_dic = make_inchidic(out)

ikeys.columns = ["InChIKey"]
ikeys.to_csv("InchiKeys.txt", quoting=csv.QUOTE_NONE, escapechar='&')

retrieve ClassyFire classifcations

In [10]:
get_classifications("InchiKeys.txt")

1000
2000
3000
4000
5000
6000


In [11]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [12]:
final.head()

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
0,1,343,4,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
1,3,343,4,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
2,20,343,4,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
3,1959,343,4,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
4,2,S1,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [13]:
final.to_csv("ClassyFireResults_Network.txt", sep = '\t', index = False)

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [14]:
len(set(list(ikeys.InChIKey)) - set(list(df.inchikey)))/len(set(list(ikeys.InChIKey)))

0.005929487179487179

which InChIKeys could not be classified?

In [15]:
set(list(ikeys.InChIKey)) - set(list(df.inchikey))

{'InChIKey=ABOXWZCNWOBUEO-VZUYHUTRSA-N',
 'InChIKey=BEMSJAOWFQJOQA-RRHRGVEJSA-N',
 'InChIKey=BHWYZGMLEVGFMX-UHFFFAOYSA-O',
 'InChIKey=BIMRUUBGKLOKIW-UHFFFAOYSA-N',
 'InChIKey=DQVGVYRSVYCJRR-FIQWRLCJSA-N',
 'InChIKey=DZUXGQBLFALXCR-BRIYLRKRSA-N',
 'InChIKey=FJKJTTCPLFXVEB-WBVITSLISA-N',
 'InChIKey=GCJSAIGGRCSHDG-VZUYHUTRSA-N',
 'InChIKey=GOHNIXDAQODZKP-IKCNDWCXSA-N',
 'InChIKey=GZBVGLNVBKBGBN-RWYGWLOXSA-N',
 'InChIKey=IKBJPCBQVRRWBQ-DZCWXNLFSA-N',
 'InChIKey=IRIIDLWTFLCEMW-RWYGWLOXSA-N',
 'InChIKey=JBYXPOFIGCOSSB-IPBYNONMSA-N',
 'InChIKey=KDCYUKBGSBLZDI-RRHRGVEJSA-N',
 'InChIKey=KEJZKSADWMARKE-UHFFFAOYSA-N',
 'InChIKey=KQXVERRYBYGQJZ-UHFFFAOYSA-N',
 'InChIKey=KWSQCBUVZQFFAT-UHFFFAOYSA-N',
 'InChIKey=LLTQYPOQKZYEMB-AKAJXFOGSA-N',
 'InChIKey=MKOKWBRPIBQYJJ-QOTIJYPWSA-N',
 'InChIKey=OMOPBJMQRZZWCK-AKAJXFOGSA-N',
 'InChIKey=PCRXUPNJCXVABN-RRHRGVEJSA-N',
 'InChIKey=PRMWQIVYOYCJQC-VIQWUECVSA-N',
 'InChIKey=PUKCSTQGOYUVRY-KEGXYFPCSA-N',
 'InChIKey=PXGNVFUWTUIRCJ-UHFFFAOYSA-N',
 'InChIKey=PXGPL

create graphML file

In [17]:
graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'FEATURE' in x][0]

In [18]:
graphML = read_graphml(graphMLfile)

In [19]:
graphML_classy = make_classyfire_graphml(graphML,final)

write graphML file

In [20]:
nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)